## Calcuate ACVF & ACF


- 자기 공분산의 경우, 기존의 표본공분산을 MVUE로 추정하면 1/(N-1)로 하듯이 1/(N-K)로 하는 부분이 있지만, 그럴 경우 표본 공분산 행렬이 non-singular이고 positive definte하지 않을 수 있다.

- 실제 응용 단계에서는 N이 굉장이 길기 때문에, 1/N 1/(N-k)의 차이는 작지만, 표본공분산이 최소분산불편추정과 같은 정확한 추정치보다, positive definte의 성질을 가지는 것이 더 중요하다.
    - ref. https://stats.stackexchange.com/questions/129052/acf-and-pacf-formula

In [ ]:
# modified from 
# https://stackoverflow.com/questions/20110590/how-to-calculate-auto-covariance-in-python
# PACF formula
# https://stats.stackexchange.com/questions/129052/acf-and-pacf-formula

import numpy as np
import re
import statsmodels.tsa.api as tsa
## Data R passengers data diff(AirPassengers)
data = """      
      Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec
1949         6   14   -3   -8   14   13    0  -12  -17  -15   14
1950   -3   11   15   -6  -10   24   21    0  -12  -25  -19   26
1951    5    5   28  -15    9    6   21    0  -15  -22  -16   20
1952    5    9   13  -12    2   35   12   12  -33  -18  -19   22
1953    2    0   40   -1   -6   14   21    8  -35  -26  -31   21
1954    3  -16   47   -8    7   30   38   -9  -34  -30  -26   26
1955   13   -9   34    2    1   45   49  -17  -35  -38  -37   41
1956    6   -7   40   -4    5   56   39   -8  -50  -49  -35   35
1957    9  -14   55   -8    7   67   43    2  -63  -57  -42   31
1958    4  -22   44  -14   15   72   56   14 -101  -45  -49   27
1959   23  -18   64  -10   24   52   76   11  -96  -56  -45   43
1960   12  -26   28   42   11   63   87  -16  -98  -47  -71   42"""
data = re.findall(r'-?\d+', data)
data = [int(i) for i in data ]
data = [i for i in data if i < 1000]
Xi = data

N = np.size(Xi)
k = 5
Xs = np.average(Xi)



def autocovariance(Xi, N, k, Xs, adjusted = False):
    autoCov = 0
    for i in np.arange(0, N-k):
        autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
    denom = (N-k) if adjusted else N
    return (1/(denom))*autoCov

def autocorrelation(Xi, N, k, Xs):
    return autocovariance(Xi, N, k, Xs) / autocovariance(Xi, N, 0, Xs)

# 
def make_toeplitz(acf_vals):
    acf_matrix = [acf_vals.copy() for i in range(len(acf_vals))]
    for i in range(0,len(acf_vals)):
        acf_matrix[i][:i+1] = acf_vals[:i+1][::-1]
        acf_matrix[i][i+1:] = acf_vals[1: len(acf_vals)-i]
    return acf_matrix


print("Autocovariance:", autocovariance(Xi, N, k, Xs, adjusted=False))
print("Autocorrelation:", autocorrelation(Xi, N, k, Xs))
print("Sample Autocovariance:", autocovariance(Xi, N, k, Xs, adjusted=True))

acf_vals = tsa.acf(Xi,nlags = k)
# use: scipy.linalg.toeplitz(acf_vals[:k])
acf_matrix = make_toeplitz(acf_vals[:k])
acf_matrix = np.array(acf_matrix)
# using left pseudo inverse
pacf = np.linalg.inv(acf_matrix.T @ acf_matrix) @ acf_matrix.T @ acf_vals[1:k+1]
# use np.linalg.solve for system of equations

print("Partial Autocovariance:", pacf[-1])
print()

import statsmodels.tsa.api as tsa

acvf_at5 = tsa.acovf(Xi, nlag = k)[-1] 
acf_at5 = tsa.acf(Xi,nlags = k)[-1]
print("Autocovariance:", acvf_at5)
# print("sample Autocovariance:", sample_acvf_at5)
print("Autocorrelation:",acf_at5)

print("Sample Autocovariance:", tsa.acovf(Xi, nlag = k, adjusted=True)[-1])
print("Partial Autocovariance:", tsa.pacf(Xi,5, 'ldb')[-1])

# Autocovariance: -106.43236439827959
# Autocorrelation: -0.09407271245807856
# Sample Autocovariance: -110.28860948517378
# Partial Autocovariance: 0.010083794316645259

# Autocovariance: -106.43236439827956
# Autocorrelation: -0.09407271245807855
# Sample Autocovariance: -110.28860948517375
# Partial Autocovariance: 0.010083794316645335